In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
from numpy import reshape
import re
import seaborn as sns
%matplotlib inline

/Users/andrewcaffey/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [383]:
os.chdir("/Users/andrewcaffey/Documents/Projects/Data/PCPP/parts/PSU/parts/")
part_list = []
comments = []
for i in os.listdir(os.getcwd()):
    a = open(i, 'r')
    #print a.read()
    b = BeautifulSoup(a)

    #basic info
    #ratings
    average_rating = b.find('span', attrs={'itemprop': 'ratingValue'})
    ratings_count = b.find('span', attrs={'itemprop':'ratingCount'})
    if (average_rating != None) & (ratings_count != None):
        ratings_dict = {'average_rating':average_rating.text, 'ratings_count':ratings_count.text}
    
    
    if b.find('h4', attrs={'class':'kind'}) != None:
        kind = b.find('h4', attrs={'class':'kind'}).text
        part_name = b.find('h1', attrs={'class':'name'}).text
        link = b.find('input', attrs={'name':'url'})['value']
        info_dict = {'Kind':kind, 'Name':part_name, 'Link': link}

        #prices
        if b.find_all('td', attrs={'class':'base'}) != None:
            price_list = b.find_all('td', attrs={'class':'base'})
            price_list = [float(x.text.strip('$')) for x in price_list]
            #average_price = sum(price_list)/len(price_list)
            price_dict = {'Prices':price_list,}

        #specs
        spec_labels = b.find('div', attrs={'class':'specs block'}).find_all('h4')
        spec_labels = [x.contents[0] for x in spec_labels]
        spec_values = str(b.find('div', attrs={'class':'specs block'}))

        vals = [x.strip().split('</h4>')[1].strip('\n').strip() for x in spec_values.split("<h4>")[1:]]
        vals[-1] = vals[-1].split('\n')[0]
        spec_values = vals

        spec_values = spec_values[0:len(spec_labels)+1]
        spec_dict = {spec_label:spec_value for spec_label, spec_value in zip(spec_labels,spec_values)}

        part_dict = dict(spec_dict.items() + info_dict.items() + price_dict.items() + ratings_dict.items())
        part_list.append(part_dict)

        #reviews
        

    #ratings
    if (average_rating != None) & (ratings_count != None):
        ratings_dict = {'average_rating':average_rating, 'ratings_count':ratings_count}
    reviews = b.find('div', attrs={'class':'part-reviews'})
    if reviews != None:
        reviews = reviews.find_all('div',attrs={'class':'part-review-block'})
        star_list = [len(reviews[x].find('ul',attrs={'class':'stars'}).find_all('li',attrs={'class':'full-star'})) for x in range(len(reviews))]

        comment_text_list = b.find_all('div', attrs={'class':'comment-message markdown'})
        comment_text_list = [comment_text_list[x].find_all('p') for x in range(len(comment_text_list))]

        comment_text_list_clean = []
        for i, x in enumerate(comment_text_list):
            comment = ""
            for y in x:
                try:
                    comment += y.contents[0] + " "
                except:
                    pass
            comment_text_list_clean.append(comment)

        

        review = zip(star_list, comment_text_list_clean)
        
        comments.append(review)

    a.close()
    b.decompose()

In [384]:
df = pd.DataFrame(part_list)

In [27]:
df.shape

(1434, 22)

In [ ]:
#cleaning
df.average_rating = [float(x) for x in df.average_rating]
df.ratings_count = [float(x) for x in df.ratings_count]

#average price
df['avg'] = [sum(x)/len(x) if len(x)>0 else 0 for x in df.Prices]

#short_link
df['short_link'] = [x.split('/')[-1] for x in df.Link]

#Wattage
df['power'] = [float(x.strip('Watts')) if x != '' else 0 for x in df.Wattage]

#Efficiency Rank Mapping
eff_rank_mapping = {u'80+ Bronze':2, u'80+ Gold':4, u'80+':1, u'80+ Titanium':6, u'80+ Platinum':5, u'-':0, u'80+ Silver':3}
df['eff_rank'] = df['Efficiency Certification'].map(eff_rank_mapping)

#price per watt
df['ppw'] = [x/y for x,y in zip(df.avg, df.power)]

In [395]:
#save df to local csv file
df.to_csv('psu_csv.csv', encoding='utf-8', index=False)